## Some preparation


In [1]:

import os
os.chdir("/Users/liutianlin/Desktop/Academics/MINDS/neuromorphic")

import numpy as np
import matplotlib.pyplot as plt
from dynapseReservoir.networks.inputsParameters import in_5_0_P
from dynapseReservoir.networks.reservoirsParameters import generalNetP 
import DYNAPSETools.dynapseSpikesGenerator as DSG


# fix a random seed
np.random.seed(in_5_0_P["seed"]) 

plt.close("all")

###  Create some types of helper events.

These helper events are not related to input spikes themselves. They are defined here to help us bookkeeping the experiment procedure. When doing analysis on collected spikes later, these events help us identify which portion of the collected spikes are the responses of the input spikes. 

We needs 5 types of helper events for the pulse experiment. 

(1) startTrigger: a spike that marks the start time of the experiment 

(2) startStimulus: a spike (slightly later than "startTrigger" in time), which marks the starting time of stimulus.

(3) startPseudoStimulus: a single spike that marks the starting time of "pseudo stimulus", i.e., the stimulus whosereceiver is not connected to the reservoir. pseudo stimulus help us prelong the collecting time after the train of "real" spikes has ended. 

(4) endStimulus: a spike that marks the end of stimulus

(5) stopTrigger: a spike that marks the end of the experiment


In [2]:
# This event indicates the starting time for the experiment
startTrigger = DSG.InputPattern(name = "startTrigger", isiBase = in_5_0_P["isiBase"])
U, C, N = generalNetP["inStartTriggerPos"]
startTrigger.single_event(virtualSourceCoreId = C,
                          neuronAddress = N,
                          coreDest = in_5_0_P["coreDest"],
                          firePeriod = in_5_0_P["startDelay"])

# This event indicates the starting time for inputing stimuli
startStimulus = DSG.InputPattern(name = "startStimulus", isiBase = in_5_0_P["isiBase"])
U, C, N = generalNetP["inTargetEncodingPos"]
startStimulus.single_event(virtualSourceCoreId = C,
                                neuronAddress = N,
                                coreDest = in_5_0_P["coreDest"],
                                firePeriod = in_5_0_P["safeDelay"]) 


# the stop of stimulus
endStimulus = DSG.InputPattern(name = "endStimulus", isiBase = in_5_0_P["isiBase"])
U, C, N = generalNetP["inEndStimulusPos"]
endStimulus.single_event(virtualSourceCoreId = C,
                         neuronAddress = N,
                         coreDest = in_5_0_P["coreDest"],
                         firePeriod = in_5_0_P["safeDelay"]) # note: keep collecting the spikes for 0.5 second after the last input stimuli

# the stop of experiment
stopTrigger = DSG.InputPattern(name = "stopTrigger", isiBase = in_5_0_P["isiBase"])
U, C, N = generalNetP["inStopTriggerPos"]
stopTrigger.single_event(virtualSourceCoreId = C,
                         neuronAddress = N,
                         coreDest = in_5_0_P["coreDest"],
                         firePeriod = in_5_0_P["stopDelay"])






###  Create some the events of input stimulus

We first write input signals as continuous singal, and convert the continuous signal into spikes


In [3]:
ecg_event = DSG.InputPattern(name = "ecg_event", isiBase = in_5_0_P["isiBase"])

##### the continuous signal (a slope y = 5 \times t) for the pulse part

# f = open('/Users/liutianlin/Desktop/Academics/MATLAB/ECG_bluebox/train_ecgInputContinuous.txt','r')
f = open('/Users/liutianlin/Desktop/Academics/MATLAB/ECG_bluebox/15min_ecgInputContinuous.txt','r')

ecgTimeSeries_y = []
for line in f.readlines():
    ecgTimeSeries_y.append(float(line.replace('\n','')))

ecgTimeSeries_y = np.array(ecgTimeSeries_y)

tStart = 0

minues = 15
tStop = minues * 60
ecgTimeSeries_t = np.linspace(tStart, tStop, len(ecgTimeSeries_y))


#### encode the continuous signal into pulse + no pulse
U, C, N1 = generalNetP["inCH0"]
U, C, N2 = generalNetP["inCH1"]



In [4]:
myThres = 0.1
ecg_event.threshold_encoder(virtualSourceCoreId = C,
                          neuronAddressUpCH = N1,  
                          neuronAddressDwCH = N2,  
                          coreDest = in_5_0_P["coreDest"],   
                          threshold = myThres,    
                          t = ecgTimeSeries_t,    
                          y = ecgTimeSeries_y,    
                          noiseVar = in_5_0_P["sig1Jitter"],      
                          initDelay = in_5_0_P["sig1Delay"])  

# Fake spikes

In [5]:
signal2 = DSG.InputPattern(name = "signal2", isiBase = in_5_0_P["isiBase"])

tSig2 = np.linspace(tStart, tStop, len(ecgTimeSeries_y)) 
ySig2 =  tSig2


U, C, N3 = generalNetP["inCH3"] # U04C00N24
U, C, N4 = generalNetP["inCH4"] # U04C00N24

signal2.threshold_encoder(virtualSourceCoreId = C,
                          neuronAddressUpCH = N3,
                          neuronAddressDwCH = N4,
                          coreDest = in_5_0_P["coreDest"],
                          threshold = 0.1,
                          t = tSig2,
                          y = ySig2,
                          noiseVar = in_5_0_P["sig2Jitter"],
                          initDelay = in_5_0_P["sig2Delay"])


In [6]:
listOfPatterns = [ecg_event, signal2]
# listOfPatterns = [signal2]


In [7]:
tupleList = []

for pattern in listOfPatterns:
    prevTime = 0
    patternTuple = []
    for event in pattern.eventList:
        patternTuple.append([event.virtualSourceCoreId, event.neuronAddress, event.coreDest, event.time + prevTime])
        prevTime += event.time
    tupleList.extend(patternTuple)   

tupleList.sort(key=lambda myList: myList[3]) # sort based on absolute time     


diffList = [0]*len(tupleList)
diffList[0] = tupleList[0][3]
for i in np.arange(1, len(tupleList)):
    diff = tupleList[i][3] - tupleList[i-1][3]
    if diff <= 10:
        diff = 10
    diffList[i] = diff # convert absolute time back to relative time

mergedPattern = DSG.InputPattern(name = "mergedPattern", isiBase = in_5_0_P["isiBase"])

for i in np.arange(len(tupleList)):
    virtualSourceCoreId = tupleList[i][0]
    neuronAddress = tupleList[i][1]
    coreDest = tupleList[i][2]
    time = diffList[i]/ 1e5 # change back to second
    mergedPattern.single_event(virtualSourceCoreId, neuronAddress, coreDest, firePeriod = time)


### Finally, Put all events togeter!

In [8]:
finalPattern = [startTrigger,
                  startStimulus,
                  mergedPattern,
                  endStimulus,
                  stopTrigger]


In [9]:
# ====================== Write events to output file in cAER folder

# the output file that reford the spikes
# fileName = "/Users/liutianlin/Desktop/Academics/MINDS/neuromorphic/caer/myStimulus/train_ecgInputContinuous_in_ecg.txt"
# fileName = "/Users/liutianlin/Desktop/Academics/MINDS/neuromorphic/caer/myStimulus/test_ecgInputContinuous_in_ecg.txt"
# fileName = "/Users/liutianlin/Desktop/Academics/MINDS/neuromorphic/caer/myStimulus/train_ecg_in.txt"
fileName = "/Users/liutianlin/Desktop/Academics/MINDS/neuromorphic/caer/myStimulus/15min_ecg_in.txt"


# DSG.write_to_file(*finalPattern, fileName = fileName)
DSG.write_to_file(*finalPattern, fileName = fileName)

# DSG.write_to_file(*[startTrigger, startStimulus, signal2, endStimulus, stopTrigger], fileName = fileName)


Checking and writing pattern startTrigger
Current pattern lenght is 1
Cumulative pattern lenght is 1. Remaining 32766 events
Pattern startTrigger succesfully written

Checking and writing pattern startStimulus
Current pattern lenght is 1
Cumulative pattern lenght is 2. Remaining 32765 events
Pattern startStimulus succesfully written

Checking and writing pattern mergedPattern
Current pattern lenght is 17117
Cumulative pattern lenght is 17119. Remaining 15648 events
Pattern mergedPattern succesfully written

Checking and writing pattern endStimulus
Current pattern lenght is 1
Cumulative pattern lenght is 17120. Remaining 15647 events
Pattern endStimulus succesfully written

Checking and writing pattern stopTrigger
Current pattern lenght is 1
Cumulative pattern lenght is 17121. Remaining 15646 events
Pattern stopTrigger succesfully written

